<a href="https://colab.research.google.com/github/Ripa-Shah/Analyzing-Social-Sentiment-Data-to-determine-correlation-with-Cryptocurrency-Price/blob/main/MIS584_Cryptocurrency_sentimentdata_and_prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
pip install pyspark praw nltk pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.6 MB/s eta 0:00:00


In [17]:
%pwd

'/content'

In [18]:
pip install tweepy praw requests pandas textblob vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.6 MB/s eta 0:00:00


In [20]:
import praw
import pandas as pd
import time
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [22]:
# List of time filters to check, ordered from newest to oldest
TIME_FILTERS = ["day", "week", "month", "year"]

# Define the start date (as before)
start_date_str = "2025-08-15 00:00:00"
start_timestamp = datetime.strptime(start_date_str, "%Y-%m-%d %H:%M:%S").timestamp()

# Initialize a set to prevent processing the same post multiple times
processed_ids = set()

In [23]:
import praw
import pandas as pd
import time
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize Reddit API
reddit = praw.Reddit(
    client_id="M9iSExe3kRkwrILVO75-CA",
    client_secret="rnuYodBu7llgTAmssHujZVUt37mrAA",
    user_agent="crypto-sentiment-app by u/MindlessEssay2919"
)

# Define cryptocurrency keywords to track
crypto_keywords = {
    "bitcoin": "BTC",
    "btc": "BTC",
    "ethereum": "ETH",
    "eth": "ETH",
    "cardano": "ADA",
    "ada": "ADA",
    "dogecoin": "DOGE",
    "doge": "DOGE",
    "solana": "SOL",
    "sol": "SOL"
}

# CSV file path
CSV_FILE = "reddit_crypto_sentiment.csv"

# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Initialize CSV file (only once)
df_init = pd.DataFrame(columns=[
    "date", "user_id", "type", "title", "cryptocurrency", "review", "sentiment_score"
])
df_init.to_csv(CSV_FILE, index=False)

print("🔄 Starting Reddit post & comment sentiment streaming...")

# ----------------------------------------------------------------------
# NEW FETCHING LOGIC
# ----------------------------------------------------------------------

records = []
print(f"Fetching posts since {start_date_str}...")

# 1. Iterate over the time filters
for time_filter in TIME_FILTERS:
    print(f"\n--- Checking Top posts for: {time_filter.upper()} ---")

    # Fetch the top posts for the current time filter (max 1000 per request)
    # The 'limit=None' tells PRAW to fetch up to its internal limit (usually 1000)
    for post in reddit.subreddit("CryptoCurrency").top(time_filter=time_filter, limit=None):
        post_timestamp = post.created_utc

        # Check 1: Has this post been processed already? (Prevents duplicates from different time filters)
        if post.id in processed_ids:
            continue

        # Check 2: Is the post within the required August 1st to Today range?
        if post_timestamp >= start_timestamp:

            # Mark the post as processed
            processed_ids.add(post.id)

            # --- Start Existing Processing Logic (Post) ---
            title = post.title
            review = post.selftext if post.selftext else ""
            user_id = str(post.author) if post.author else "Anonymous"
            date = datetime.utcfromtimestamp(post_timestamp).strftime("%Y-%m-%d %H:%M:%S")

            # Detect which crypto is mentioned
            text_lower = (title + " " + review).lower()
            found_coin = None
            # ... (Rest of your crypto keyword detection logic)
            for keyword, symbol in crypto_keywords.items():
                if keyword in text_lower:
                    found_coin = symbol
                    break
            if not found_coin:
                continue

            # Sentiment for post
            sentiment = analyzer.polarity_scores(title + " " + review)["compound"]
            records.append([date, user_id, "post", title, found_coin, review, sentiment])
            print(f"✅ Found Post ({time_filter}): {date} | {found_coin} | Sentiment: {sentiment:+.3f}")

            # --- Start Existing Comment Processing Logic ---
            post.comments.replace_more(limit=0)
            for comment in post.comments.list()[:10]:
                # ... (Your existing comment processing and sentiment logic goes here)
                comment_text = comment.body
                comment_user = str(comment.author) if comment.author else "Anonymous"
                comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S")

                if not comment_text or comment_text in ["[deleted]", "[removed]"]:
                    continue

                comment_lower = comment_text.lower()
                comment_coin = found_coin
                for keyword, symbol in crypto_keywords.items():
                    if keyword in comment_lower:
                        comment_coin = symbol
                        break

                comment_sent = analyzer.polarity_scores(comment_text)["compound"]
                records.append([comment_date, comment_user, "comment", title, comment_coin, comment_text, comment_sent])

        elif time_filter == "year" and post_timestamp < start_timestamp:
            # Only break out of the loop if we're on the longest time filter and found old data.
            # Shorter filters ("day", "week") will naturally stop finding posts as they go too far back.
            print(f"--- Stopping fetch: Hit post older than {start_date_str} on the 'year' filter. ---")
            break

# ----------------------------------------------------------------------
# END NEW FETCHING LOGIC
# ----------------------------------------------------------------------

# --- Final Saving Logic (Run once after all fetching is complete) ---
if records:
    df = pd.DataFrame(records, columns=["date", "user_id", "type", "title", "cryptocurrency", "review", "sentiment_score"])
    df.to_csv(CSV_FILE, mode="a", header=False, index=False)
    print(f"\n✅ FINAL SAVE: Collected and saved {len(records)} unique entries to {CSV_FILE}")

print("Processing complete.")

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



🔄 Starting Reddit post & comment sentiment streaming...
Fetching posts since 2025-08-15 00:00:00...

--- Checking Top posts for: DAY ---


/tmp/ipython-input-2245392545.py:72: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date = datetime.utcfromtimestamp(post_timestamp).strftime("%Y-%m-%d %H:%M:%S")
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



✅ Found Post (day): 2025-11-15 04:51:36 | BTC | Sentiment: -0.296


/tmp/ipython-input-2245392545.py:96: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S")
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



...

✅ Found Post (month): 2025-10-25 01:47:13 | ETH | Sentiment: +0.981


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



✅ Found Post (month): 2025-11-04 07:57:52 | ETH | Sentiment: +0.000


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.




--- Checking Top posts for: YEAR ---
--- Stopping fetch: Hit post older than 2025-08-15 00:00:00 on the 'year' filter. ---

✅ FINAL SAVE: Collected and saved 1882 unique entries to reddit_crypto_sentiment.csv
Processing complete.


In [1]:
!pip install pyspark

In [2]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("MIS584_Spark_Basics")
sc = SparkContext(conf = conf)

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MIS584_Spark_Basics").getOrCreate()

In [4]:

# you can get a spark context from a spark session
sc = spark.sparkContext

In [ ]:
#download the dataset crypto_prices_historical.csv and reddit_crypto_Sentiment.csv

In [31]:
# create a spark dataframe from a pandas dataframe
import pandas as pd

## first, read the external csv file into a pandas dataframe
crypto_prices = pd.read_csv('crypto_prices_historical.csv')

## then, convert the pandas dataframe to a spark dataframe
df = spark.createDataFrame(crypto_prices)
df.show(n=10)

## we can also convert a spark dataframe to a pandas dataframe
pdf = df.toPandas()
print(pdf.head(20))

+---------------+-------+-----------+
|      timestamp| symbol|      price|
+---------------+-------+-----------+
|8/15/2025 17:00|BTC-USD|117678.0078|
|8/15/2025 18:00|BTC-USD|117189.3281|
|8/15/2025 19:00|BTC-USD|117005.4922|
|8/15/2025 20:00|BTC-USD|117355.7578|
|8/15/2025 21:00|BTC-USD|117377.7891|
|8/15/2025 22:00|BTC-USD|117777.8281|
|8/15/2025 23:00|BTC-USD|117416.2891|
| 8/16/2025 0:00|BTC-USD|117834.2734|
| 8/16/2025 1:00|BTC-USD|117899.3203|
| 8/16/2025 2:00|BTC-USD| 117597.875|
+---------------+-------+-----------+
only showing top 10 rows

          timestamp   symbol        price
0   8/15/2025 17:00  BTC-USD  117678.0078
1   8/15/2025 18:00  BTC-USD  117189.3281
2   8/15/2025 19:00  BTC-USD  117005.4922
3   8/15/2025 20:00  BTC-USD  117355.7578
4   8/15/2025 21:00  BTC-USD  117377.7891
5   8/15/2025 22:00  BTC-USD  117777.8281
6   8/15/2025 23:00  BTC-USD  117416.2891
7    8/16/2025 0:00  BTC-USD  117834.2734
8    8/16/2025 1:00  BTC-USD  117899.3203
9    8/16/2025 2:00  B

In [34]:
# create a spark dataframe from a pandas dataframe
import pandas as pd

## first, read the external csv file into a pandas dataframe
df_reddit_crypto = pd.read_csv('reddit_crypto_sentiment.csv')

## then, convert the pandas dataframe to a spark dataframe
df_reddit_crypto = spark.createDataFrame(pdf)
df_reddit_crypto.show(n=5)

## we can also convert a spark dataframe to a pandas dataframe
pdf_reddit = df_reddit_crypto.toPandas()
print(pdf_reddit.head(20))

+-------------------+-----------------+-------+--------------------+--------------+--------------------+---------------+
|               date|          user_id|   type|               title|cryptocurrency|              review|sentiment_score|
+-------------------+-----------------+-------+--------------------+--------------+--------------------+---------------+
|2025-11-15 04:51:36|       GreedVault|   post|Jeffrey Epstein W...|           BTC|                 NaN|         -0.296|
|2025-11-15 04:51:37|    AutoModerator|comment|Jeffrey Epstein W...|           BTC|It looks like thi...|         0.9063|
|2025-11-15 08:41:41|Large-Example1665|comment|Jeffrey Epstein W...|           BTC|He should have wo...|        -0.7579|
|2025-11-15 05:00:15|    coinfeeds-bot|comment|Jeffrey Epstein W...|           BTC|tldr; Jeffrey Eps...|         0.8979|
|2025-11-15 09:33:02|  Clear_Hawk_6187|comment|Jeffrey Epstein W...|           BTC|Do we know his Re...|         0.6124|
+-------------------+-----------

In [35]:
from pyspark.sql.types import *

# create a spark dataframe from a csv file with inferred schema:
df_reddit_crypto = spark.read.csv('reddit_crypto_sentiment.csv', header=True, inferSchema=True) # Spark will automatically detect the data type of each column.
df_reddit_crypto.show()

# print the schema (column names and types of a dataframe)
print('Inferred Schema of the DataFrame:')
df_reddit_crypto.printSchema()

# create a spark dataframe from a csv file with specified schema:
schema = StructType([
    StructField('date', TimestampType(), True),
    StructField('user_id', StringType(), True),
    StructField('tyoe', StringType(), True),
    StructField('title', StringType(), True),
    StructField('cryptocurrency', StringType(), True),
    StructField('review',StringType(),True),
    StructField('sentiment_score',DoubleType(),True)
])
df_reddit_crypto = spark.read.csv('reddit_crypto_sentiment.csv', header=True, schema=schema)
print('Specified Schema of the DataFrame:')
df_reddit_crypto.printSchema()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+
|                date|             user_id|                type|               title|      cryptocurrency|              review|sentiment_score|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+
| 2025-11-15 04:51:36|          GreedVault|                post|Jeffrey Epstein W...|                 BTC|                NULL|         -0.296|
| 2025-11-15 04:51:37|       AutoModerator|             comment|Jeffrey Epstein W...|                 BTC|It looks like thi...|           NULL|
|Please note that ...|             suggest|            advocate|     instruct others| or ask for help ...|                NULL|           NULL|
|Note:  Tax discus...|                NULL|                NULL|                NULL|                NULL|                NULL|         

In [36]:
# get the number of rows for a dataframe
print()
print('Number of rows of the DataFrame:', df_reddit_crypto.count())

# get the number of columns and column ames for a dataframe
print()
print('Number of columns of the DataFrame:', len(df_reddit_crypto.columns))
print('Column names of the DataFrame:', df_reddit_crypto.columns)


Number of rows of the DataFrame: 3452

Number of columns of the DataFrame: 7
Column names of the DataFrame: ['date', 'user_id', 'tyoe', 'title', 'cryptocurrency', 'review', 'sentiment_score']


In [37]:
# drop duplicates from a dataframe
print('Number of rows before dropping duplicates:', df_reddit_crypto.count())
df_reddit_crypto.dropDuplicates().count()
print('Number of rows after dropping duplicates:', df_reddit_crypto.dropDuplicates().count())

Number of rows before dropping duplicates: 3452
Number of rows after dropping duplicates: 2702


In [45]:
#print statistics of data frame
df_reddit_crypto.describe()

DataFrame[summary: string, user_id: string, tyoe: string, title: string, cryptocurrency: string, review: string, sentiment_score: string]

In [53]:
#rename column tyoe to type

df_reddit_crypto = df_reddit_crypto.withColumnRenamed("tyoe", "type")

# Display the new schema to confirm the change
df_reddit_crypto.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- cryptocurrency: string (nullable = true)
 |-- review: string (nullable = true)
 |-- sentiment_score: double (nullable = true)



In [59]:
from pyspark.sql.functions import current_timestamp, to_timestamp


In [70]:
#fill missing values with keyword missing and for sentiment_score ->0.0
df_reddit_crypto_filled = df_reddit_crypto.na.fill(value=0.0, subset=['sentiment_score'])
#df_reddit_crypto.na.fill(current_timestamp(), subset=["date"])
df_reddit_crypto_missing = df_reddit_crypto_filled.na.fill(value='missing', subset=['cryptocurrency','title','review','user_id','type'])
#df_reddit_crypto.na.fill(value='1/1/1930', subset=['date']).show(100)

In [71]:
df_reddit_crypto_missing.show(n=5)

+-------------------+-------------+---------+--------------------+--------------------+--------------------+---------------+---------------+
|               date|      user_id|     type|               title|      cryptocurrency|              review|sentiment_score|sentiment_label|
+-------------------+-------------+---------+--------------------+--------------------+--------------------+---------------+---------------+
|2025-11-15 04:51:36|   GreedVault|     post|Jeffrey Epstein W...|                 BTC|             missing|         -0.296|       Negative|
|2025-11-15 04:51:37|AutoModerator|  comment|Jeffrey Epstein W...|                 BTC|It looks like thi...|            0.0|        Neutral|
|               NULL|      suggest| advocate|     instruct others| or ask for help ...|             missing|            0.0|        Neutral|
|               NULL|      missing|  missing|             missing|             missing|             missing|            0.0|        Neutral|
|            

In [73]:

# apply the filter transformation on a dataframe
print('Reddit posts and comments positive sentiment score:')
df_reddit_crypto_missing.filter(df_reddit_crypto['sentiment_score'] > 0.0)
df_reddit_crypto_missing.show(n=5)
print('Reddit posts and comments negative sentiment score:')
df_reddit_crypto_missing.filter(df_reddit_crypto['sentiment_score'] < 0.0)
df_reddit_crypto_missing.show(n=5)
print('Reddit posts and comments neutral sentiment score:')
df_reddit_crypto_missing.filter(df_reddit_crypto['sentiment_score'] == 0.0)
df_reddit_crypto_missing.show(n=5)



Reddit posts and comments positive sentiment score:
+-------------------+-------------+---------+--------------------+--------------------+--------------------+---------------+---------------+
|               date|      user_id|     type|               title|      cryptocurrency|              review|sentiment_score|sentiment_label|
+-------------------+-------------+---------+--------------------+--------------------+--------------------+---------------+---------------+
|2025-11-15 04:51:36|   GreedVault|     post|Jeffrey Epstein W...|                 BTC|             missing|         -0.296|       Negative|
|2025-11-15 04:51:37|AutoModerator|  comment|Jeffrey Epstein W...|                 BTC|It looks like thi...|            0.0|        Neutral|
|               NULL|      suggest| advocate|     instruct others| or ask for help ...|             missing|            0.0|        Neutral|
|               NULL|      missing|  missing|             missing|             missing|             mi

In [74]:
#Add additional column called sentiment_label
from pyspark.sql.functions import col, when
POSITIVE_THRESHOLD = 0.1
NEGATIVE_THRESHOLD = -0.1

df_reddit_crypto_missing = df_reddit_crypto_missing.withColumn(
    "sentiment_label",
    when(col("sentiment_score") > POSITIVE_THRESHOLD, "Positive")
    .when(col("sentiment_score") < NEGATIVE_THRESHOLD, "Negative")
    .otherwise("Neutral")
)
df_reddit_crypto_missing.show(n=10)


+-------------------+--------------------+---------+--------------------+--------------------+--------------------+---------------+---------------+
|               date|             user_id|     type|               title|      cryptocurrency|              review|sentiment_score|sentiment_label|
+-------------------+--------------------+---------+--------------------+--------------------+--------------------+---------------+---------------+
|2025-11-15 04:51:36|          GreedVault|     post|Jeffrey Epstein W...|                 BTC|             missing|         -0.296|       Negative|
|2025-11-15 04:51:37|       AutoModerator|  comment|Jeffrey Epstein W...|                 BTC|It looks like thi...|            0.0|        Neutral|
|               NULL|             suggest| advocate|     instruct others| or ask for help ...|             missing|            0.0|        Neutral|
|               NULL|             missing|  missing|             missing|             missing|             missi

In [81]:
# group a dataframe by certain columns and calculate group statistics
from pyspark.sql.functions import max, min, sum, avg, stddev, round, count

print('Top total number of posts and comments posted by a userid:')
df_reddit_count = df_reddit_crypto_missing.groupby('user_id').agg(
    count('review').alias('total_reviews'),
    count('title').alias('total_titles')
)
df_reddit_count_sort = df_reddit_count.sort(col('total_reviews').desc(), col('total_titles').desc())
df_reddit_count_sort.show(n=10)

Top total number of posts and comments posted by a userid:
+--------------------+-------------+------------+
|             user_id|total_reviews|total_titles|
+--------------------+-------------+------------+
|             missing|          657|         657|
|              DYOR."|           82|          82|
|       coinfeeds-bot|           82|          82|
|         DryMyBottom|           41|          41|
|            partymsl|           35|          35|
|           kirtash93|           30|          30|
|       AutoModerator|           28|          28|
|                 0.0|           27|          27|
|               KIG45|           26|          26|
| and this action ...|           21|          21|
+--------------------+-------------+------------+
only showing top 10 rows



In [93]:
from pyspark.sql.functions import col, countDistinct
df_reddit_crypto_count =  df_reddit_crypto_missing.groupBy("cryptocurrency", "type").agg(
    countDistinct("user_id").alias("unique_user_count")
)

df_reddit_crypto_count.select(['unique_user_count', 'cryptocurrency', 'type'])\
.sort('unique_user_count', 'cryptocurrency', ascending=[False, False]).show(n=15)

+-----------------+--------------------+-------+
|unique_user_count|      cryptocurrency|   type|
+-----------------+--------------------+-------+
|              865|                 BTC|comment|
|              337|             missing|missing|
|              270|                 ETH|comment|
|               68|                 SOL|comment|
|               55|                 BTC|   post|
|               34|                 ETH|   post|
|               23|                 ADA|comment|
|               10| there is no seco...|comment|
|               10| Bitcoin Will Be ...|comment|
|                7|             missing|    0.0|
|                5|                 SOL|   post|
|                4|             missing| 0.2732|
|                3|                 ADA|   post|
|                2|             missing| 0.6908|
|                2|             missing| 0.7783|
+-----------------+--------------------+-------+
only showing top 15 rows

